# Title Placeholder

# First Section
Config, starting variables and query

In [0]:

# Putting this on a separate cell to avoid cleaning variables by the interpreter restart
# Ensure gspread is installed
try:
    import gspread
    print("gspread is already installed.")
except ImportError:
    print("gspread is not installed. Installing...")
    %pip install gspread
    

In [0]:

from pyspark.sql import SparkSession # Allows the Spark session to work with Databricks DB
from datetime import date, datetime, timedelta # Allows working with dates
from decimal import Decimal # Allows casting Decimal type to string or float for JSON conversion
import pandas as pd # Dataframe manipulation
import gspread # Read and write GSheets



# Create a Spark session
spark = SparkSession.builder.appName("NamePlaceholder").getOrCreate()


# current_date = date(2024, 5, 31)
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# End of Month date of previous month
end_of_previous_month = first_day_of_current_month - timedelta(days=1)

# For this particular extraction, the date is always the End of Month date of the previous month
extract_date = end_of_previous_month.strftime("%Y-%m-%d") # Formating date to 'YYYY-MM-DD'

# report_date will be the extract date
report_date_string = extract_date


#        ################ IMPORTANT!!!!!! ################
# Aside from getting a service account to run the Google API for Gsheets, we
# need to give editor permission to the service account on the desired GSheet using 
# the email we can find in the json generated when creating said service account.

# Full Path -> /Workspace/Users/username@something.com/folderName/serviceAccountPlaceholder.json
# URL -> URL_Placeholder

SHEET_ID = 'SheetIDPlaceholder' # Sheet Name Placeholder
# WORKSHEET_NAME = 'WorksheetNamePlaceholder'
WORKSHEET_ID = IDNumberPlaceholder # Using worksheet ID to avoid error due to name changes of the worksheets

creds = gspread.service_account("/Workspace/Users/username@something.com/folderName/serviceAccountPlaceholder.json")

spreadsheet = creds.open_by_key(SHEET_ID)
# worksheet = spreadsheet.worksheet(WORKSHEET_NAME)
worksheet = spreadsheet.get_worksheet_by_id(WORKSHEET_ID)

In [0]:

# Using f string to declare the query so we can replace '{extract_date}' with the declared value

QUERY = f'''

SELECT * FROM TableName
WHERE date = '{extract_date}'


'''

# Second Section
This section will query our SQL database to generate the current month data up to the current date.
The result will be a Pandas dataframe with dates casted as strings.

In [0]:
# Specify catalog
spark.sql("USE CATALOG catalogPlaceholder")

# Running Query and storing results in a Spark DataFrame
spark_df_new_data =  spark.sql(QUERY)

# Convert Spark DataFrame to Pandas DataFrame (if using Spark)
pandas_df_new_data = spark_df_new_data.toPandas()

# Convert all date columns in the DataFrame to string format
pandas_df_new_data = pandas_df_new_data.applymap(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, date) else x)


# Third Section (__NOT IN USE__)
In this section, we will prepare the data for an Incremental Load:
1) ~~Query the GSheet to retrive the currently registered data~~
2) ~~Compare it with the data obtained from our SQL database using the "report_date" column~~

In [0]:


############## INCREMENTAL LOAD #################
# At the moment, I prefer to load all the available data of the current month to avoid
# differences due to retroactive changes but I'll leave the logic for the future
#################################################

# # Specify the range of columns to read (e.g., 'A:D' for columns A to D)
# range_to_read = 'A:H'

# # Get data for the specified range
# # this will produce a list of lists (each list inside the list will be a row)
# gsheet_data = worksheet.get(range_to_read)

# # Turning into a dataframe to manipulate data
# gsheet_data_df = pd.DataFrame(gsheet_data[1:], columns=gsheet_data[0])

# # Filtering df through difference using column "report_date" as reference
# filtered_df = pandas_df_new_data[~pandas_df_new_data["report_date"].isin(gsheet_data_df["report_date"])]

# # Convert Decimal values to float before creating the list of lists
# filtered_df = filtered_df.applymap(lambda x: float(x) if isinstance(x, Decimal) else x)

# # We will create a list of lists without headers:
# list_new_filtered_data = filtered_df.values.tolist()

#################################################



# Fourth Section
In this section, we will:
1) Check data to write and cast Decimal to float for JSON conversion
2) Turn the df into a list of lists without headers
3) Find the row that match the "report_date" or first empty row <- IMPORTANT! 'YYYY-MM-DD'
4) Write to the GSheet

In [0]:

# Convert Decimal values to float before creating the list of lists
pandas_df_new_data = pandas_df_new_data.applymap(lambda x: float(x) if isinstance(x, Decimal) else x)

# We will create a list of lists (required to convert to JSON) without headers
list_new_data = pandas_df_new_data.values.tolist()

# /////////////////////NOTE////////////////////
# Due to data being small enough, I'll just overwrite the data everytime it loads instead of
# doing batch updates

# # Get all values from column "A" (assuming column "A" is "report_date" and
# # contains dates in 'YYYY-MM-DD' format)
# col_values = worksheet.col_values(1)

# # Find the row index of the report_date
# if report_date_string in col_values:
#   # Add 1 because GSheets is 1-indexed and list indexing is 0-indexed
#   row_index = col_values.index(report_date_string) + 1
# else: # If we don't get a match then first empty row
#   row_index = len(col_values) + 1


# Write the list of lists to the Google Sheet starting on the cell that match
# the first day of the current month or the first empty row.

# raw="false" will simulate manual input in the GSheet
# worksheet.update(list_new_data, f"A{row_index}", raw="false") # NOTE: uncomment this when using batch updates
worksheet.update(list_new_data, f"A2", raw="false")

# Adding an empty row to the end so we can append new data without hitting
# the "exceed grid limit" error
worksheet.add_rows(300)


print("Work Done!")

In [0]:
list_new_data

In [0]:
pandas_df_new_data.info()